In [1]:
# Used for selecting the 6 best features per cluster. 
# Assumes data is vectorized + clustered.
# We're using simple f_regression score of each variable vs. the ALSFRS_score, and take the best 6. 

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import vectorizer_beta
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn import linear_model


In [33]:
proact_data = pd.read_csv('../train_data.csv', sep = '|', index_col=False)
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col=False)
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col=False)
print proact_data.shape, slope.shape, clusters.shape
X = pd.merge(clusters, proact_data, on = "SubjectID")
Y = pd.merge(X, slope, on = "SubjectID")

X.head()

(1138647, 6) (1777, 2) (213, 2)


,SubjectID,cluster,form_name,feature_name,feature_value,feature_unit,feature_delta
0,103126,0,Demographic,Gender,M,NaN,0.0
1,103126,0,Demographic,Age,26,NaN,0.0
2,103126,0,Demographic,Race,Unknown,NaN,0.0
3,103126,0,ALSHX,onset_delta,-770,NaN,0.0
4,103126,0,ALSHX,diag_delta,-527,NaN,0.0


In [28]:
from vectorizer_beta import * 
print func_per_feature

{'weight': <function timeseries_feature_slope_reduced at 0x000000000C4579E8>, 'temperature': <function timeseries_feature_slope_reduced at 0x000000000C4579E8>, 'bp_systolic': <function timeseries_feature_slope_reduced at 0x000000000C4579E8>, 'Gender': <function scalar_feature_to_dummies at 0x000000000C419BA8>, 'BMI': <function timeseries_feature_last_value at 0x000000000C457198>, 'respiratory_rate': <function timeseries_feature_slope_reduced at 0x000000000C4579E8>, 'pulse': <function timeseries_feature_slope_reduced at 0x000000000C4579E8>, 'bp_diastolic': <function timeseries_feature_slope_reduced at 0x000000000C4579E8>, 'Race': <function scalar_feature_to_dummies at 0x000000000C419BA8>, 'height': <function timeseries_feature_last_value at 0x000000000C457198>}


In [32]:
import vectorizer_beta
selector_per_cluster = {}

for c in clusters['cluster'].unique():
    seg_X, seg_Y = X[X['cluster'] == c], Y[Y['cluster'] == c]
    score_per_feature = {}
    for feature_name, func in func_per_feature.iteritems():
        seg_X_fam = func_per_feature[feature_name](seg_X, feature_name)
        seg_X_fam = seg_X_fam.fillna(seg_X_fam.mean())
        regr = linear_model.LinearRegression()
        regr.fit(seg_X_fam, seg_Y['ALSFRS_slope'])
        score_per_family[family] = np.mean((regr.predict(seg_X_fam) - seg_Y['ALSFRS_slope']) ** 2)
    print c, score_per_family

ValueError: Found arrays with inconsistent numbers of samples: [  200 60053]

In [5]:
def calc(x):
    selector = selector_per_cluster[x['cluster']]
    d = {"feature_ " + str(i): v for i, v in enumerate(selector.transform(x)[0])}
    d['features_list'] = ';'.join(cur_X.columns[selector.get_support()])
    d['cluster'] = int(x['cluster'])
    return pd.Series(d)

for t in ['train', 'test']:
    cur_data = pd.read_csv('../' + t + '_data_vectorized.csv', sep = '|', index_col='SubjectID')
    cur_clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col='SubjectID')
    cur_X = pd.merge(cur_data, cur_clusters, left_index = True, right_index = True)
    res = cur_X.apply(calc, axis = 1)
    res.to_csv('../' + t + '_selected_features.csv',sep='|')
    